# Banking Deserts

In [ ]:
# Dependencies
from census import Census
from config import census_key
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")


# Create an instance of the Census library
c = Census(census_key, year=2019)

## 1. Retrieve data from the U.S. Census using the Census Python library and the preconfigured labels

References:

* Review the following page to review the Python library documentation: <https://github.com/CommerceDataService/census-wrapper>

* Review the following page to learn more about the data labels: <https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b>

In [ ]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
census_data = c.acs5.get(
    (
        "B01003_001E",
        "B17001_002E"
    ),
    {'for': 'zip code tabulation area:*'}
)

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column renaming
census_pd = census_pd.rename(
    columns = {
        "B01003_001E": "Population",
        "B17001_002E": "Poverty Count",
        "zip code tabulation area": "Zipcode"
    }
)

# Add a Poverty Rate column (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

# Configure the final DataFrame
census_pd = census_pd[
    [
        "Zipcode",
        "Population",
        "Poverty Rate"
    ]
]

# Display DataFrame length and sample data
print(f"Number of rows in the DataFrame: {len(census_pd)}")
census_pd.head()

## 2. Load the `zip_bank_data.csv` file into a DataFrame. Next, use Pandas to merge this data set with the Census data that you retrieved along the zip code.

In [ ]:
# Import the zip bank data. Use dtype="object" to ensure all data is read in accurately.
zip_code_pd = pd.read_csv(
    "../Resources/zip_bank_data.csv",
    dtype = "object",
    encoding = "utf-8"
)

# Display sample data
zip_code_pd.head()

In [ ]:
# Merge the two data sets along zip code
census_data_complete = pd.merge(
    zip_code_pd,
    census_pd,
    how = "left",
    on = ["Zipcode", "Zipcode"]
)

# Remove rows with missing data
census_data_complete = census_data_complete.dropna()

# Transform latitude and longitude to float
census_data_complete["Lat"] = census_data_complete["Lat"].astype(float)
census_data_complete["Lng"] = census_data_complete["Lng"].astype(float)

# Display sample data
census_data_complete.head()

## 3. Use GeoViews to create a poverty rate map.

* Use the "Poverty Rate" column to set the point's size. Recall to use the `scale` parameter to modify the size appearance.

* Use the "Zipcode" column to set the point's colour.

* Read the HoloViews documentation and learng how you can use the `hover_cols` parameter to add additional information to the tooltip of a point. Add the "Address" and the "Bank Count" columns.

In [ ]:
# Configure the map
poverty_rate_map = census_data_complete.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Poverty Rate",
    scale = 2,
    color = "Zipcode",
    hover_cols = ["Address", "Bank Count"]
)

# Display the map plot
poverty_rate_map

## 4. Compute and print the summary statistics for "Poverty Rate", "Bank Count", and "Population"

In [ ]:
# Mean, median, mode for Poverty Rate
poverty_mean = round(census_data_complete["Poverty Rate"].astype("float").mean(), 2)
poverty_median = round(census_data_complete["Poverty Rate"].astype("float").median(), 2)
poverty_mode = round(census_data_complete["Poverty Rate"].astype("float").mode(), 2)

print(f"Poverty Rate Mean: {poverty_mean}")
print(f"Poverty Rate Median {poverty_median}")
print(f"Poverty Rate mode {poverty_mode}")

In [ ]:
# Mean, median, mode for Bank Count
bank_mean = round(census_data_complete["Bank Count"].astype("float").mean(), 2)
bank_median = round(census_data_complete["Bank Count"].astype("float").median(), 2)
bank_mode = round(census_data_complete["Bank Count"].astype("float").mode(), 2)

print(f"Bank Count Mean: {bank_mean}")
print(f"Bank Count Median {bank_median}")
print(f"Bank Count mode {bank_mode}")

In [ ]:
# Mean, median, mode for Population
population_mean = round(census_data_complete["Population"].astype("float").mean(), 2)
population_median = round(census_data_complete["Population"].astype("float").median(), 2)
population_mode = round(census_data_complete["Population"].astype("float").mode(), 2)

print(f"Population Mean: {population_mean}")
print(f"Population Median {population_median}")
print(f"Population mode {population_mode}")

## 5. Create a scatter plot with linear regression for **bank count** vs. **poverty rate**.

* Be sure `NaN` values are dropped from the DataFrame.

* Plot the data points.

* Plot the linear regression line.

* Print the R<sup>2</sup> value.

In [ ]:
## Convert to floats and store Poverty Rate and Bank Count as x and y values
x_values = census_data_complete["Poverty Rate"].astype("float")
y_values = census_data_complete["Bank Count"].astype("float")

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.xlabel('Poverty Rate')
plt.ylabel('Bank Count')

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

## 6. Analysis.

Write a sentence describing your findings. Were they what you expected? What other factors could be at play?

* There is a very weak correlation between poverty rates and bank counts. Keep in mind that linear regression will not consider other factors such as population or size of the city.